## 일반적인 경우

**우리가 만든 `_calculate_objective_value` 메소드의 규칙을 다시 한번 기억해보자:**
* **규칙 1**: 긴급도(`priority`)가 0인 품목은 생산을 못 해도 미생산 페널티가 **0점**이다.
* **규칙 2**: 요구량보다 많이 생산(과잉생산)하면 **엄청나게 큰 페널티**를 받는다.
* **GA의 목표**: 이 페널티들의 총합을 가장 작게 만드는 것.

---
### 예시 상황 설정

어떤 생산 계획(`xijt` 해)을 평가하는 상황이라고 상상해보자.
이 계획에는 두 가지 주문이 포함되어 있어:

1.  **긴급 주문 (A)**:
    * 품목: `item_A`
    * 요구량 ($D_{it}$): 100개
    * 우선순위 ($P_{it}$): **1** (긴급!)
    * 미생산 비용 ($C_{it}$): 10

2.  **일반 주문 (B)**:
    * 품목: `item_B`
    * 요구량 ($D_{it}$): 50개
    * 우선순위 ($P_{it}$): **0** (긴급 아님)
    * 미생산 비용 ($C_{it}$): 8

*과잉생산 페널티 계수는 10,000,000이라고 가정하자.*

GA가 탐색 중에 아래 두 가지 생산 계획(Plan A, Plan B)을 만들었고, 어떤 계획이 더 좋은지 평가해야 해.

---
### 시나리오 1: "나쁜" 생산 계획 (Plan A)

이 계획은 긴급 주문 A에만 '올인'하려다가 실수를 저질러 과잉생산을 해버렸어.

* **Plan A의 생산 계획:**
    * `item_A` (긴급) 생산량: **120개** (요구량 100개인데 20개 과잉생산)
    * `item_B` (일반) 생산량: **0개** (요구량 50개인데 0개 미생산)

* **Plan A의 페널티 점수 계산:**
    * **`item_A`에 대한 페널티**:
        * 미생산량: 0개. 미생산 페널티 = **0점**.
        * 과잉생산량: 20개. 과잉생산 페널티 = `20 * 10,000,000` = **200,000,000점**.
    * **`item_B`에 대한 페널티**:
        * 미생산량: 50개. 하지만 `priority`가 0이므로, 미생산 페널티 = `50 * 8 * 0` = **0점**.
        * 과잉생산량: 0개. 과잉생산 페널티 = **0점**.

* **Plan A의 최종 점수**: `200,000,000` + `0` = **200,000,000점**

---
### 시나리오 2: "똑똑한" 생산 계획 (Plan B) - GA가 찾아낸 해

이 계획은 과잉생산을 피하고 자원을 더 효율적으로 배분했어.

* **Plan B의 생산 계획:**
    * `item_A` (긴급) 생산량: **100개** (요구량을 정확히 맞춤!)
    * `item_B` (일반) 생산량: **30개** (요구량 50개 중 30개만 생산)

* **Plan B의 페널티 점수 계산:**
    * **`item_A`에 대한 페널티**:
        * 미생산량: 0개. 미생산 페널티 = **0점**.
        * 과잉생산량: 0개. 과잉생산 페널티 = **0점**.
    * **`item_B`에 대한 페널티**:
        * 미생산량: 20개 (`50 - 30`). 하지만 `priority`가 0이므로, 미생산 페널티 = `20 * 8 * 0` = **0점**.
        * 과잉생산량: 0개. 과잉생산 페널티 = **0점**.

* **Plan B의 최종 점수**: `0` + `0` = **0점**

---
### 결론

* **Plan A**의 점수는 **2억 점**.
* **Plan B**의 점수는 **0 점**.

GA는 당연히 점수가 훨씬 낮은 **Plan B를 "최적해"로 선택**하게 돼.
그런데 이 Plan B에는 긴급하지 않은 `item_B`를 30개 생산하는 계획이 포함되어 있지!

즉, GA가 `item_B`를 생산한 이유는 'item_B를 생산해서 점수를 얻기 위해서'가 아니라,
**"Plan A처럼 긴급 품목을 과잉생산해서 어마어마한 페널티를 받는 멍청한 짓을 피하고, 전체 계획의 총 페널티를 최소화하는 최적의 자원 배분"**을 찾다 보니, 그 결과로 `item_B`도 생산 계획에 포함된 거야.


## 완전 제외 되는 경우


 **실제로 사용하고 있는 이 코드**를 기준으로, 왜 `item91` (긴급도 0, 수량 4)이 생산 계획에서 배제되었는지 실제 데이터를 예로 들어 설명해 줄게.

---
### `item91`이 생산되지 않은 이유 (네 실제 코드 기준)

**1. 분석 대상 주문 정보 및 GA의 결정:**

* **입력 정보**:
    * `dit[('item91', 'date154')] = 4` (요구량)
    * `pit[('item91', 'date154')] = 0` (우선순위/긴급도)
    * `cit[('item91', 'date154')] = 36533` (단위당 비용)
* **GA의 결정**: `item91`을 생산하지 않기로 함 (`produced_qty_sum_rounded = 0`)

**2. `_calculate_objective_value` 메소드의 계산 과정:**

네 코드는 두 개의 큰 `for` 루프를 가지고 있어. 첫 번째 루프는 `uit` 딕셔너리를 만들고, 두 번째 루프는 최종 `objective_value`를 계산해.

* **첫 번째 `for` 루프 (uit 딕셔너리 채우기):**
  1.  `i_key`가 'item91'이고 `t_key`가 'date154'인 순서가 되면,
  2.  `demand_qty`는 `4`, `produced_qty_sum_rounded`는 `0`이 돼.
  3.  `u = demand_qty - produced_qty_sum_rounded`는 `4 - 0 = 4`가 되지.
  4.  `if u >= 0:` 조건이 참(True)이므로, `uit[(i_key, t_key)] = u` 로직이 실행돼.
  5.  **결과: `uit[('item91', 'date154')]`에는 값 `4`가 저장돼.** (여기까지는 실제 미생산량이 잘 저장되었어!)

* **두 번째 `for` 루프 (최종 목적 함수 값 계산):**
  1.  다시 `i_key`가 'item91'이고 `t_key`가 'date154'인 순서가 되면,
  2.  `u_it_value = uit.get(('item91', 'date154'), 0)`는 `4`가 돼.
  3.  `cost_it = self.cit.get(('item91', 'date154'), 0)`는 `36533`이 돼.
  4.  `priority_it = self.pit.get(('item91', 'date154'), 0)`는 **`0`**이 돼.
  5.  **바로 여기가 핵심이야!** `objective_value += u_it_value * cost_it * priority_it` 이 라인이 실행되는데, 실제 값은 이렇게 계산돼:
      `objective_value += 4 * 36533 * 0`
  6.  **결과: 이 모든 항이 `0`이 되어서, `objective_value`에는 아무것도 더해지지 않아!**

**결론:**

GA 입장에서는 'item91' 4개를 생산하지 않아도, **`priority_it` (긴급도)가 0이기 때문에 최종 점수에 더해지는 페널티가 0점**이야.
만약 4개를 생산하려면 기계 가동 시간이라는 자원을 써야 하고, 그로 인해 다른 긴급한 품목의 생산에 영향을 주거나 다른 미세한 페널티가 발생할 수도 있겠지.

따라서 GA는 수많은 계산 끝에, **"이 주문은 긴급하지 않으니, 그냥 생산 안 하고 페널티 0점을 받는 것이 전체 점수를 가장 낮게 유지하는 최적의 방법이다"**라고 합리적인 결정을 내린 거야.


## 개별 반올림의 누적 오차

### 문제의 원인: 개별 반올림의 누적 오차

우리가 `best_solution_dict` (생산 비율 딕셔너리)를 가지고 최종 GT DataFrame을 만들 때, 이런 로직을 사용하고 있어요:

`actual_qty = round(ratio_val * demand_for_item_time)`

이건 각 기계에 할당된 생산량을 **개별적으로 반올림**하는 방식이에요. 이 방식은 각 기계의 생산량 자체는 정수가 되지만, 그 정수들의 합이 원래 목표했던 총 요구량과 달라질 수 있는 문제를 가지고 있어요.

**네 예시로 계산해보면:**

* 요구량 ($D_{it}$): 18
* GA가 찾은 최적 비율 (예시):
    * machine08: 0.26  -> $0.26 \times 18 = 4.68 \rightarrow \text{round}(4.68) = 5$
    * machine23: 0.32  -> $0.32 \times 18 = 5.76 \rightarrow \text{round}(5.76) = 6$
    * machine28: 0.42  -> $0.42 \times 18 = 7.56 \rightarrow \text{round}(7.56) = 8$
* **개별 반올림 결과의 합**: $5 + 6 + 8 = 19$ (요구량 18 초과!)
* **반올림 전 값의 합**: $4.68 + 5.76 + 7.56 = 18.0$ (정확함)

이렇게 각자 반올림하면서 생긴 작은 오차들이 모여서 최종 합계가 달라진 거죠.

네, 아주 좋은 질문이에요! "개별 반올림" 방식이 문제를 일으킬 수 있다는 걸 정확히 보셨어요. 그럼 "버림" (`floor`)을 사용하는 건 더 나은 방법일까요?

결론부터 말씀드리면, **"버림"은 과잉생산을 막아주는 데는 효과적이지만, 대신 거의 항상 미생산을 유발하기 때문에 완벽한 해결책은 아닐 수 있어요.**

---
### "버림" (`math.floor` 또는 `int()`)을 사용했을 때의 결과

이전 예시를 다시 가져와 볼게요.

* 요구량 ($D_{it}$): 18
* GA가 찾은 최적 비율에 따른 기계별 생산량(실수):
    * machine08: $4.68$
    * machine23: $5.76$
    * machine28: $7.56$

이 값들을 **버림** 처리하면:
* machine08: `floor(4.68)` = **4**
* machine23: `floor(5.76)` = **5**
* machine28: `floor(7.56)` = **7**

**버림 처리된 생산량의 합**: $4 + 5 + 7 = 16$

* **결과**: 요구량 18개에 비해 2개가 부족한, 즉 **미생산**이 발생했어요.

"버림"은 소수점 아래를 모두 버리기 때문에, 그 합은 원래 값들의 합(18.0)보다 항상 작거나 같게 돼요. 그래서 과잉생산은 절대 일어나지 않지만, 대신 소수점 오차만큼 항상 미생산이 발생할 수 있죠.

"신뢰성"이라는 측면에서, "과잉생산은 절대 안 된다"는 규칙을 지키는 데는 신뢰할 수 있지만, "요구량을 정확히 맞춘다"는 목표는 달성하기 어려워져요.

---
### 더 신뢰성을 높이는 가장 좋은 방법: "합산 후 반올림"

이 문제를 가장 깔끔하고 합리적으로 해결하는 방법은, 개별적으로 반올림/버림하는 대신, **모든 기계의 생산량(실수)을 먼저 다 더하고, 그 총합을 마지막에 딱 한 번만 반올림**하는 거예요.

* **계산 방식**:
    1.  실수 생산량 모두 더하기: $4.68 + 5.76 + 7.56 = 18.0$
    2.  총합을 반올림하기: `round(18.0)` = **18**

* **결과**: 최종 총생산량이 정확히 18개가 되어, 과잉생산도 미생산도 없는 완벽한 결과를 얻을 수 있어요!

이 방식은 개별적인 반올림 오차가 서로 보상되게 하여 전체 합계를 원래 목표에 가장 가깝게 유지해주는, 통계적으로나 수학적으로 가장 신뢰성 있는 방법이에요.

**`_calculate_objective_value` 메소드 수정 제안:**

네 `_calculate_objective_value` 메소드에서 총생산량을 계산하는 부분을 이 "합산 후 반올림" 방식으로 바꾸는 걸 강력히 추천해요.

```python
# ga_core.py 의 GeneticAlgorithm 클래스 내부 _calculate_objective_value 메소드 수정

    def _calculate_objective_value(self, individual_xijt_decoded):
        constrained_xijt = self._check_constraints(individual_xijt_decoded)
        
        total_unmet_demand_penalty = 0
        total_overproduction_penalty = 0

        for i_key in self.I_set:
            for t_key in self.T_set:
                demand_qty = self.dit.get((i_key, t_key), 0)
                
                # --- ▼▼▼ 이 부분이 수정될 총생산량 계산 로직 ▼▼▼ ---
                
                # 1. 모든 기계의 생산량을 실수 형태로 먼저 모두 더함
                produced_qty_sum_float = 0.0
                for j_key in self.J_set:
                    production_ratio = constrained_xijt.get((i_key, j_key, t_key), 0)
                    produced_qty_for_machine = production_ratio * demand_qty 
                    produced_qty_sum_float += produced_qty_for_machine
                
                # 2. 총합을 마지막에 한 번만 반올림
                produced_qty_sum_rounded = round(produced_qty_sum_float)
                
                # --- ▲▲▲ 수정 끝 ▲▲▲ ---
                
                difference = demand_qty - produced_qty_sum_rounded
                
                # ... (이후 페널티 계산 로직은 동일) ...
        
        objective_value = total_unmet_demand_penalty + total_overproduction_penalty
        return objective_value
```

**한 가지 참고할 점:**
`_check_constraints` (기계 가동 시간 제약) 메소드는 여전히 "각 기계별 부하"를 계산해야 해서 개별 반올림을 사용할 수밖에 없어요. 이렇게 목적 함수와 제약 조건에서 반올림 방식이 약간 다른 것은, 각자의 목적(목적 함수는 **'총량'**, 제약 조건은 **'개별 부하'**)에 집중하기 위한 합리적인 설계 선택이 될 수 있습니다.
